使用許諾条件

①	設計一次エネルギー消費量算定法に基づく計算方法をPythonによりコーディングし、Jupyter notebook形式で保存したファイル（以下、本ファイルと呼ぶ）の著作権は、一般社団法人日本サステナブル建築協会に帰属します。
Copyright©2019. 一般社団法人日本サステナブル建築協会
All rights reserved. 

②	本ファイルは、住戸の設計一次エネルギー消費量算定方法に係る「平成28年省エネルギー基準に準拠したエネルギー消費性能の評価に関する技術情報（住宅）2.1算定方法（第四章第七節 温水暖房）」に示す計算方法に基づき、国土交通省国土技術政策総合研究所・国立研究開発法人建築研究所、株式会社ユーワークス、株式会社住環境計画研究所が作成したものです。

③	本ファイルは、計算方法に示す計算過程の確認及び調査研究等に活用することを目的として作成されたものであり、一般社団法人日本サステナブル建築協会は著作権について放棄はしませんが、使用する各人の自己責任において仕様を変更することを妨げるものではありません。ただし、本ファイルによる計算結果を住宅の省エネルギー基準で規定する設計一次エネルギー消費量の結果として公的な申請に用いることはできません。

④	本ファイルを変更するかしないかを問わず、以下の条件を満たす場合に限り、頒布および使用が許可されます。
1.	頒布物の付属ドキュメント等の資料に、上記の著作権表示、本条件一覧、および下記免責条項を含めること。
2.	書面による特別の許可なしに、本ファイルから派生した製品の宣伝または販売の促進に、一般社団法人日本サステナブル建築協会、国土技術政策総合研究所・国立研究開発法人建築研究所および株式会社ユーワークス、株式会社住環境計画研究所の名前またはコントリビューターの名前を使用してはならない。

⑤	本ファイルは、著作権者およびコントリビューターによって「現状のまま」提供されており、明示黙示を問わず、商業的な使用可能性、および特定の目的に対する適合性に関する暗黙の保証も含め、またそれに限定されない、いかなる保証もありません。著作権もコントリビューターも、事由のいかんを問わず、損害発生の原因いかんを問わず、かつ責任の根拠が契約であるか厳格責任であるか（過失その他の）不法行為であるかを問わず、仮にそのような損害が発生する可能性を知らされていたとしても、本ファイルの使用によって発生した（代用品または代用サービスの調達、使用の喪失、データの喪失、利益の喪失、業務の中断も含め、またそれに限定されない）直接損害、間接損害、偶発的な損害、特別損害、懲罰的損害、または結果損害について、一切の責任を負わないものとします。



In [ ]:
import nbimporter
import numpy as np
import s04_07_h_max_capacity_of_hot_water_floor_heating as appendix_H

## 付録A　石油温水暖房機及び石油給湯温水暖房機 / Appendix A Oil hot water heater and oil hot water heater hot water heater

### A.1 記号及び単位 / Symbol and unit

省略

ommision

### A.2 エネルギー消費量 / Energy consumption

#### A.2.1 灯油消費量 / Kerosene consumption

$$\large
E_{K,hs} = \left\{ \begin{array}{ll}
    0&(Q_{out,H,hs} = 0)\\ 
    \frac{Q_{out,H,hs} + Q_{body}}{e_{ex}} &(Q_{out,H,hs} > 0)\\ 
    \end{array}
\right.
$$

$E_{K,hs}$：1 時間当たりの温水暖房用熱源機の灯油消費量(MJ/h) / Kerosene consumption of heat source unit for hot water heating per hour  
$Q_{out,H,hs}$：1 時間当たりの温水暖房用熱源機の暖房出力(MJ/h) / Heating output of the heat source machine for hot water heating per hour  
$Q_{body}$：1 時間当たりの温水暖房用熱源機の筐体放熱損失(MJ/h) / Heat dissipation loss of the case of the heat source unit for hot water heating per hour  
$e_{ex}$：1 時間平均の温水暖房用熱源機の熱交換効率 / Heat exchange efficiency of one hour average hot water heating heat source machine

In [ ]:
def calc_E_K_hs(Q_out_H_hs, e_rtd, hs_type, Theta_SW_hs, Theta_RW_hs, region, A_A, A_MR, A_OR, mode_MR, mode_OR,
                has_MR_hwh, has_OR_hwh):
    """ 温水暖房用熱源機の灯油消費量 (1)

    :param Q_out_H_hs: 熱源機暖房出力 (MJ/h)
    :type Q_out_H_hs: ndarray
    :param e_rtd: 当該給湯機の効率
    :type e_rtd: float
    :param hs_type: 温水暖房用熱源機の種類
    :type hs_type: str
    :param Theta_SW_hs: 温水暖房用熱源機の往き温水温度
    :type Theta_SW_hs: ndarray
    :param Theta_RW_hs: 1時間平均の熱源機の戻り温水温度 (℃)
    :type Theta_RW_hs: ndarray
    :param region: 省エネルギー地域区分
    :type region: int
    :param A_A: 床面積の合計 (m2)
    :type A_A: float
    :param A_MR: 主たる居室の床面積 (m2)
    :type A_MR: float
    :param A_OR: その他の居室の床面積 (m2)
    :type A_OR: float
    :param mode_MR: 主たる居室の運転モード 'い', 'ろ', 'は'
    :type mode_MR: str
    :param mode_OR: その他の居室の運転モード 'い', 'ろ', 'は'
    :type mode_OR: str
    :param has_MR_hwh: 温水暖房の放熱器を主たる居室に設置する場合はTrue
    :type has_MR_hwh: bool
    :param has_OR_hwh: 温水暖房の放熱器をその他の居室に設置する場合はTrue
    :type has_OR_hwh: bool
    :return: 温水暖房用熱源機の灯油消費量
    :rtype: ndarray
    """
    # 定格試験時の温水暖房用熱源機の筐体放熱損失 (4)
    Q_body_rtd = get_Q_body_rtd(hs_type)

    # 温水暖房熱源機の定格能力
    q_rtd_hs = calc_q_rtd_hs(region, A_A, A_MR, A_OR, mode_MR, mode_OR, has_MR_hwh, has_OR_hwh)

    # 1時間平均の温水暖房用熱源機の熱交換効率 (3)
    e_ex = calc_e_ex(e_rtd, hs_type, Theta_RW_hs, q_rtd_hs, Q_body_rtd)

    # 温水暖房用熱源機の筐体放熱損失 (2)
    Q_body = get_Q_body(hs_type, Theta_SW_hs)

    # 温水暖房用熱源機の灯油消費量 (1)
    E_K_hs = (Q_out_H_hs + Q_body) / e_ex
    E_K_hs[Q_out_H_hs == 0] = 0

    return E_K_hs


当該住戸の石油温水暖房機が石油従来型暖房機の場合

When the oil hot water heater of the dwelling unit concerned is a petroleum conventional heater

$$\large
Q_{body} = 234 \times 3600 \times 10^{-6}
$$

当該住戸の石油温水暖房機が石油潜熱回収型暖房機の場合

When the oil hot water heater of the relevant dwelling unit is an oil latent heat recovery type heater

$$\large
Q_{body} = (5.3928 \times \theta_{SW,hs} - 71.903) \times 3600 \times 10^{-6}
$$

$\theta_{SW,hs}$：1 時間平均の温水暖房用熱源機の往き温水温度(℃) / One hour average hot water temperature of the hot water source heater for hot water heating

In [ ]:
def get_Q_body(hs_type, Theta_SW_hs):
    """ 温水暖房用熱源機の筐体放熱損失 (2)

    :param hs_type: 温水暖房用熱源機の種類
    :type hs_type: str
    :param Theta_SW_hs: 温水暖房用熱源機の往き温水温度
    :type Theta_SW_hs: ndarray
    :return: 温水暖房用熱源機の筐体放熱損失
    :rtype: ndarray
    """
    if hs_type in ['石油従来型暖房機', '石油従来型温水暖房機', '石油従来型給湯温水暖房機', '不明']:
        # (2a)
        return [234 * 3600 * 10 ** (-6)] * 24 * 365
    elif hs_type in ['石油潜熱回収型暖房機', '石油潜熱回収型温水暖房機', '石油潜熱回収型給湯温水暖房機']:
        # (2b)
        return (5.3928 * Theta_SW_hs - 71.903) * 3600 * 10 ** (-6)
    else:
        raise ValueError(hs_type)


$$\large
e_{ex} = e_{rtd} \times f_{rtd} \times \frac{q_{rtd,hs} \times 3600 \times 10^{-6} + Q_{body,rtd}}{q_{rtd,hs} \times 3600 \times 10^{-6}}
$$

$e_{rtd}$：温水暖房用熱源機の定格効率 / Rated efficiency of the hot water heating heat source machine  
$f_{rtd}$：1 時間平均の定格効率を補正する係数 / Factor to correct rated efficiency of 1 hour average  
$q_{rtd,hs}$：温水暖房用熱源機の定格能力(W) / Rated capacity of hot water heating heat source machine  
$Q_{body,rtd}$：1 時間当たりの定格試験時の温水暖房用熱源機の筐体放熱損失(MJ/h) / Heat dissipation loss of the case of the heat source unit for hot water heating at the rated test per hour

In [ ]:
def calc_e_ex(e_rtd, hs_type, Theta_RW_hs, q_rtd_hs, Q_body_rtd):
    """ 1時間平均の温水暖房用熱源機の熱交換効率 (3)

    :param e_rtd: 当該給湯機の効率
    :type e_rtd: float
    :param hs_type: 温水暖房用熱源機の種類
    :type hs_type: str
    :param Theta_RW_hs: 1時間平均の熱源機の戻り温水温度 (℃)
    :type Theta_RW_hs: ndarray
    :param q_rtd_hs: 温水暖房用熱源機の定格能力 (W)
    :type q_rtd_hs: float
    :param Q_body_rtd: 1時間当たりの定格試験時の温水暖房用熱源機の筐体放熱損失（MJ/h）
    :type Q_body_rtd: ndarray
    :return: 1時間平均の温水暖房用熱源機の熱交換効率 (3)
    :rtype: ndarray
    """
    # 1時間平均の定格効率を補正する係数
    f_rtd = get_f_rtd(hs_type, Theta_RW_hs)

    return e_rtd * f_rtd * (q_rtd_hs * 3600 * 10 ** (-6) + Q_body_rtd) / (q_rtd_hs * 3600 * 10 ** (-6))


当該住戸の石油熱源機が従来型石油熱源機の場合

When the oil heat source machine of the relevant dwelling unit is a conventional oil heat source machine

$$\large
Q_{body,rtd} = 234 \times 3600 \times 10^{-6}
$$

当該住戸の石油熱源機が潜熱回収型石油熱源機の場合

In the case where the oil heat source of the dwelling unit is a latent heat recovery type oil heat source

$$\large
Q_{body,rtd} = (5.3928 \times \theta_{SW,hs,rtd} - 71.903) \times 3600 \times 10^{-6}
$$

$\theta_{SW,hs,rtd}$：温水暖房用熱源機の定格試験時の往き温水温度(℃) / Hot water temperature at the time of rating test of heat source equipment for hot water heating

In [ ]:
def get_Q_body_rtd(hs_type):
    """ 定格試験時の温水暖房用熱源機の筐体放熱損失 (4)

    :param hs_type: 温水暖房用熱源機の種類
    :type hs_type: str
    :return: 定格試験時の温水暖房用熱源機の筐体放熱損失 (4)
    :rtype: ndarray
    """
    if hs_type in ['石油従来型暖房機', '石油従来型温水暖房機', '石油従来型給湯温水暖房機']:
        # (4a)
        return 234 * 3600 * 10 ** (-6) * np.ones(24*365)
    elif hs_type in ['石油潜熱回収型暖房機', '石油潜熱回収型温水暖房機', '石油潜熱回収型給湯温水暖房機']:

        # 温水暖房用熱源機の定格試験時の往き温水温度
        Theta_SW_hs_rtd = get_Theta_SW_hs_rtd()

        # (4b)
        return (5.3928 * Theta_SW_hs_rtd - 71.903) * 3600 * 10 ** (-6)
    else:
        raise ValueError(hs_type)

当該住戸の石油熱源機が従来型石油熱源機の場合

When the oil heat source machine of the relevant dwelling unit is a conventional oil heat source machine

$$\large
f_{rtd} = 0.946
$$

当該住戸の石油熱源機が潜熱回収型石油熱源機の場合

In the case where the oil heat source of the dwelling unit is a latent heat recovery type oil heat source

$$\large
f_{rtd} = \left\{ \begin{array}{ll}
    0.9768 &(\theta_{RW,hs} \geq 46.5, \theta_{RW,hs,rtd} \geq 46.5)\\ 
    \frac{(-0.0023 \times \theta_{RW,hs} + 1.014)}{0.907} \times 0.9768 &(\theta_{RW,hs} < 46.5, \theta_{RW,hs,rtd} \geq 46.5)\\ 
    \frac{0.907}{(-0.0023 \times \theta_{RW,hs,rtd} + 1.014)} \times 0.9768 &(\theta_{RW,hs} \geq 46.5, \theta_{RW,hs,rtd} < 46.5)\\ 
    \frac{(-0.0023 \times \theta_{RW,hs} + 1.014)}{(-0.0023 \times \theta_{RW,hs,rtd} + 1.014)} \times 0.9768 &(\theta_{RW,hs} < 46.5, \theta_{RW,hs,rtd} < 46.5)\\ 
    \end{array}
\right.
$$

$\theta_{RW,hs}$：1 時間平均の温水暖房用熱源機の戻り温水温度(℃) / One hour average return hot water temperature of the hot water heating heat source machine  
$\theta_{RW,hs,rtd}$：温水暖房用熱源機の定格試験時の戻り温水温度(℃) / Return hot water temperature at rated test of hot water heating heat source machine

In [ ]:
def get_f_rtd(hs_type, Theta_RW_hs):
    """ 1時間平均の定格効率を補正する係数 (5)

    :param hs_type: 温水暖房用熱源機の種類
    :type hs_type: str
    :param Theta_RW_hs: 1時間平均の熱源機の戻り温水温度 (℃)
    :type Theta_RW_hs: ndarray
    :return: 1時間平均の定格効率を補正する係数 (5)
    :rtype: ndarray
    """
    f_rtd = np.zeros(24 * 365)

    if hs_type in ['石油従来型暖房機', '石油従来型温水暖房機', '石油従来型給湯温水暖房機']:
        # (5a)
        f_rtd[:] = 0.946
    elif hs_type in ['石油潜熱回収型暖房機', '石油潜熱回収型温水暖房機', '石油潜熱回収型給湯温水暖房機']:
        # (5b)

        Theta_RW_hs_rtd = get_Theta_RW_hs_rtd()

        # 条件1
        f1 = np.logical_and(Theta_RW_hs >= 46.5, Theta_RW_hs_rtd >= 46.5)
        f_rtd[f1] = 0.9768

        # 条件2
        f2 = np.logical_and(Theta_RW_hs < 46.5, Theta_RW_hs_rtd >= 46.5)
        f_rtd[f2] = (-0.0023 * Theta_RW_hs[f2] + 1.014) / 0.907 * 0.9768

        # 条件3
        f3 = np.logical_and(Theta_RW_hs >= 46.5, Theta_RW_hs_rtd < 46.5)
        f_rtd[f3] = 0.907 / (-0.0023 * Theta_RW_hs[f3] + 1.014) * 0.9768

        # 条件4
        f4 = np.logical_and(Theta_RW_hs < 46.5, Theta_RW_hs_rtd < 46.5)
        f_rtd[f4] = (-0.0023 * Theta_RW_hs[f4] + 1.014) / (-0.0023 * Theta_RW_hs_rtd + 1.014) * 0.9768

    else:
        raise ValueError(hs_type)

    return f_rtd


当該住戸の石油温水暖房機が潜熱回収型石油温水暖房機の場合、定格効率$e_{rtd}$は0.91とし、温水暖房用熱源機の定格試験時の往き温水温度$\theta_{SW,hs,rtd}$は70℃、温水暖房用熱源機の定格試験時の戻り温水温度$\theta_{RW,hs,rtd}$は50℃とする。 

If the oil water heater of the dwelling unit is a latent heat recovery type oil water heater, the rated efficiency $ e_{rtd} $ is 0.91, the temperature $\theta_{SW,hs,rtd}$ of the hot water at the time of the rating test of the heat source for hot water heating is 70°C, for hot water heating The return hot water temperature $\theta_{RW, hs, rtd}$ at the time of the rating test of the heat source equipment is 50°C.

In [ ]:
def get_e_rtd_default(hs_type):
    """ 定格効率(規定値)

    :param hs_type: 温水暖房用熱源機の種類
    :type hs_type: str
    :return: 定格効率(規定値)
    :rtype: float
    """
    if hs_type in ['石油従来型暖房機', '石油従来型温水暖房機', '石油従来型給湯温水暖房機']:
        return 0.82
    elif hs_type in ['石油潜熱回収型暖房機', '石油潜熱回収型温水暖房機', '石油潜熱回収型給湯温水暖房機']:
        return 0.91
    else:
        raise ValueError(hs_type)

In [ ]:
def get_Theta_SW_hs_rtd():
    """ 温水暖房用熱源機の定格試験時の往き温水温度

    :return: 温水暖房用熱源機の定格試験時の往き温水温度
    :rtype: float
    """
    return 70


In [ ]:
def get_Theta_RW_hs_rtd():
    """ 温水暖房熱源機の定格試験時の戻り温水温度

    :return: 温水暖房熱源機の定格試験時の戻り温水温度
    :rtype: float
    """
    return 50


In [ ]:
def calc_q_rtd_hs(region, A_A, A_MR, A_OR, mode_MR, mode_OR, has_MR_hwh, has_OR_hwh):
    """ 温水暖房熱源機の定格能力

    :param region: 省エネルギー地域区分
    :type region: int
    :param A_A: 床面積の合計 (m2)
    :type A_A: float
    :param A_MR: 主たる居室の床面積 (m2)
    :type A_MR: float
    :param A_OR: その他の居室の床面積 (m2)
    :type A_OR: float
    :param mode_MR: 主たる居室の運転モード 'い', 'ろ', 'は'
    :type mode_MR: str
    :param mode_OR: その他の居室の運転モード 'い', 'ろ', 'は'
    :type mode_OR: str
    :param has_MR_hwh: 温水暖房の放熱器を主たる居室に設置する場合はTrue
    :type has_MR_hwh: bool
    :param has_OR_hwh: 温水暖房の放熱器をその他の居室に設置する場合はTrue
    :type has_OR_hwh: bool
    :return: 温水暖房熱源機の定格能力
    :rtype: float
    """
    # 付録Hに定める温水暖房用熱源機の最大能力 q_max_hs に等しい
    return appendix_H.calc_q_max_hs(region, A_A, A_MR, A_OR, mode_MR, mode_OR, has_MR_hwh, has_OR_hwh)

#### A.2.2 消費電力量 / Power consumption

$$\large
E_{E,hs} = E_{E,hs,pmp} + E_{E,hs,fan}
$$

$E_{E,hs}$：1 時間当たりの温水暖房用熱源機の消費電力量(kWh/h) / Power consumption of the hot water heating heat source unit per hour  
$E_{E,hs,pmp}$：1 時間当たりの温水暖房用熱源機の送水ポンプの消費電力量(kWh/h) / Power consumption of the water supply pump of the heat source unit for hot water heating per hour  
$E_{E,hs,fan}$：1 時間当たりの温水暖房用熱源機の排気ファンの消費電力量(kWh/h) / Power consumption of exhaust fan of heat source unit for hot water heating per hour  

In [ ]:
def calc_E_E_hs(hs_type, r_WS_hs, E_K_hs):
    """ 温水暖房用熱源機の消費電力量 (6)

    :param hs_type: 温水暖房用熱源機の種類
    :type hs_type: str
    :param r_WS_hs: 1時間平均の温水暖房用熱源機の温水供給運転率
    :type r_WS_hs: ndarray
    :param E_K_hs: 1時間当たりの温水暖房用熱源機の灯油消費量（MJ/h）
    :type E_K_hs: ndarray
    :return: 温水暖房用熱源機の消費電力量
    :rtype: ndarray
    """
    # 温水暖房用熱源機の送水ポンプの消費電力量
    E_E_hs_pump = calc_E_E_hs_pmp(hs_type, r_WS_hs)

    # 温水暖房用の熱源機の排気ファンの消費電力量
    E_E_hs_fan = calc_E_E_hs_fan(E_K_hs)

    return E_E_hs_pump + E_E_hs_fan

$$\large
E_{E,hs,pmp} = P_{hs,pmp} \times r_{WS,hs} \times 10^{-3}
$$

$P_{hs,pmp}$：温水暖房用熱源機の送水ポンプの消費電力(W) / Power consumption of water pump of heat source machine for hot water heating  
$r_{WS,hs}$：1 時間平均の温水暖房用熱源機の温水供給運転率 / 1 hour average hot water supply operation rate of the hot water heating heat source machine

In [ ]:
def calc_E_E_hs_pmp(hs_type, r_WS_hs):
    """ 温水暖房用熱源機の送水ポンプの消費電力量 (7)

    :param hs_type: 温水暖房用熱源機の種類
    :type hs_type: str
    :param r_WS_hs: 1時間平均の温水暖房用熱源機の温水供給運転率
    :type r_WS_hs: ndarray
    :return: 温水暖房用熱源機の送水ポンプの消費電力量
    :rtype: ndarray
    """
    # 温水暖房用熱源機の送水ポンプの消費電力
    P_hs_pmp = get_P_hs_pmp(hs_type)

    return P_hs_pmp * r_WS_hs * 10 ** (-3)

温水暖房用熱源機の送水ポンプの消費電力$P_{hs,pmp}$は、当該住戸の石油温水暖房機が石油従来型温水暖房機の場合は90(W)とし、当該住戸の石油温水暖房機が石油潜熱回収型温水暖房機の場合は70（W）とする。 

The power consumption $ P_{hs, pmp} $ of the water supply pump of the hot water heating heat source machine is 90 (W) if the oil hot water heater of the dwelling unit is a petroleum conventional hot water heater, and the oil hot water heating of the dwelling unit is If the machine is an oil latent heat recovery type hot water heater, it shall be 70 (W).

In [ ]:
def get_P_hs_pmp(hs_type):
    """ 温水暖房用熱源機の送水ポンプの消費電力

    :param hs_type: 温水暖房用熱源機の種類
    :type hs_type: str
    :return: 温水暖房用熱源機の送水ポンプの消費電力
    :rtype: float
    """
    if hs_type in ['石油従来型暖房機', '石油従来型温水暖房機', '石油従来型給湯温水暖房機']:
        return 90
    elif hs_type in ['石油潜熱回収型暖房機', '石油潜熱回収型温水暖房機', '石油潜熱回収型給湯温水暖房機']:
        return 70
    else:
        raise ValueError(hs_type)


$$\large
E_{E,hs,fan} = E_{K,hs} \times \gamma \times 10^{3} \div 3600
$$

$E_{K,hs}$：1 時間当たりの温水暖房用熱源機の灯油消費量(MJ/h) / Kerosene consumption of heat source unit for hot water heating per hour  
$\gamma$：排気ファンの効率 / Exhaust fan efficiency  

In [ ]:
def calc_E_E_hs_fan(E_K_hs):
    """ 温水暖房用の熱源機の排気ファンの消費電力量 (8)

    :param E_K_hs: 1時間当たりの温水暖房用熱源機の灯油消費量（MJ/h）
    :type E_K_hs: ndarray
    :return: 温水暖房用の熱源機の排気ファンの消費電力量 (8)
    :rtype: ndarray
    """
    # 排気ファンの効率
    gamma = get_gamma()

    return E_K_hs * gamma * 10 ** 3 / 3600

排気ファンの効率$\gamma$は0.003とする。

The exhaust fan efficiency $ \gamma $ is 0.003.

In [ ]:
def get_gamma():
    """ 排気ファンの効率

    :return: 排気ファンの効率
    :rtype: float
    """
    return 0.003


#### A.2.3 ガス消費量 / Gas consumption

1 時間当たりの温水暖房用熱源機のガス消費量$E_{G,hs}$は0とする。 

The gas consumption $ E_{G, hs} $ of the hot water heating heat source unit per hour is assumed to be zero.

In [ ]:
def get_E_G_hs():
    """ ガス消費量

    :return: ガス消費量
    :rtype: ndarray
    """
    return np.zeros(24 * 365)


#### A.2.4 その他の燃料による一次エネルギー消費量 / Primary energy consumption by other fuels

1 時間当たりの温水暖房用熱源機のその他の燃料による一次エネルギー消費量$E_{M,hs}$は0とする。 

Primary energy consumption $ E_{M, hs} $ by the other fuels of the hot water heating heat source unit per hour is assumed to be zero.

In [ ]:
def get_E_M_hs():
    """ その他の燃料による一次エネルギー消費量

    :return: その他の燃料による一次エネルギー消費量
    :rtype: ndarray
    """
    return np.zeros(24 * 365)


### A.3 最大暖房出力 / Maximum heating output

$$\large
Q_{max,H,hs} = q_{rtd,hs} \times 3600 \div 10^{6} 
$$

In [ ]:
def get_Q_max_H_hs(q_rtd_hs):
    """ 最大暖房出力 (9)

    :param q_rtd_hs: 温水暖房用熱源機の定格能力 (W)
    :type q_rtd_hs: float
    :return: 最大暖房出力
    :rtype: ndarray
    """
    return np.ones(24*365) * q_rtd_hs * 3600 / (10 ** 6)
